In [1]:
import serial as ser
import numpy as np
from math import pi, cos, sin, radians
import time

### Forward Kinematic (Vorherige Aufgabe)

In [4]:
def forward_kinematic_braccio(joint_angles):
    ''' param: joints: current position of all the joints of the braccio '''
    ''' return: current position'''

    result = np.eye(4)
    
    ## initial translation
    result = result @ translation([281, 0, 3])
    
    ## Joint 0 (Base b)
    result = result @ rotate_z(90 - joint_angles[0]) @ translation([0, 0, 71]) 
    
    ## Joint 1 (Shoulder s)
    result = result @ rotate_y(90 - joint_angles[1]) @ translation([0, 0, 126]) 
    
    ## Joint 2 (Elbow e)
    result = result @ rotate_y(90 - joint_angles[2]) @ translation([0, 0, 124]) 
    
    ## Joint 3 (Wrist tilt v)
    result = result @ rotate_y(90 - joint_angles[3]) @ translation([0, 0, 60])

    ## Joint 4 (Wrist rotate w)
    result = result @ rotate_z(90 - joint_angles[4]) @ translation([0, 0, 134]) 
    
    return result

def translation(trans):
    return np.array([
        [1,0,0,trans[0]],
        [0,1,0,trans[1]],
        [0,0,1,trans[2]],
        [0,0,0,1]
    ])

def rotate_z(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad), -sin(rad), 0, 0],
        [sin(rad), cos(rad), 0, 0],
        [0,0,1,0],
        [0,0,0,1]
    ])

def rotate_y(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad),0, sin(rad), 0],
        [0,1,0,0],
        [-sin(rad),0, cos(rad), 0],
        [0,0,0,1]
    ])

def forward_kinematic_braccio_fixed(joint_angles):
    b = joint_angles[0]
    s = 110
    e = joint_angles[1]
    v = joint_angles[2]
    w = 90
    g = 73
    joint_angles_with_fixed = [b, s, e, v, w, g]
    return forward_kinematic_braccio(joint_angles_with_fixed)

In [5]:
result = forward_kinematic_braccio_fixed([60, 160, 160])[0:3,3]
print(result)

[  78.82944746 -116.72322293   10.10090179]


### Rückwärts Kinematik

In [6]:
def F(x_soll, joint_angles):
    transformation_matrix = forward_kinematic_braccio_fixed(joint_angles)
    x_ist = transformation_matrix[0:3,3]
    return x_ist - x_soll

def dF_theta1(x_soll, joint_angles):
    h = 1E-4
    new_joint_angles = [joint_angles[0]+ h, joint_angles[1], joint_angles[2]]
    return (F(x_soll, new_joint_angles) - F(x_soll, joint_angles)) / h

def dF_theta2(x_soll, joint_angles):
    h = 1E-4
    new_joint_angles = [joint_angles[0], joint_angles[1] + h, joint_angles[2]]
    return (F(x_soll, new_joint_angles) - F(x_soll, joint_angles)) / h

def dF_theta3(x_soll, joint_angles):
    h = 1E-4
    new_joint_angles = [joint_angles[0], joint_angles[1], joint_angles[2] + h]
    return (F(x_soll, new_joint_angles) - F(x_soll, joint_angles)) / h

In [7]:
def jaccobi(x_soll, joint_angles):
    return np.stack((
        dF_theta1(x_soll, joint_angles),
        dF_theta2(x_soll, joint_angles),
        dF_theta3(x_soll, joint_angles)
    ), axis=-1)

def inv_jaccobi(x_soll, joint_angles):
    return np.linalg.pinv(jaccobi(x_soll, joint_angles))

In [8]:
def backward_kinematic_braccio(x_soll, theta_0, max_iter=1000, error=1, alpha=1):
    
    theta_i = theta_0
    
    for i in range(max_iter):
        transformation_matrix = forward_kinematic_braccio_fixed(theta_i)
        x_ist = transformation_matrix[0:3,3]
        diff = np.linalg.norm(x_soll - x_ist)
                
        if diff < error:
            return theta_i
        
        j_inv = inv_jaccobi(x_soll, theta_i)
        theta_i = theta_i - alpha * j_inv.dot(F(x_soll, theta_i))
        
    raise Exception("Sorry, no solution found") 

In [9]:
def linear_backward(x_soll, theta_0, splits=10):
    joints = []
    positions = []
    coordinates = forward_kinematic_braccio_fixed(theta_0)[0:3,3]
    positions.append(coordinates)
    print(coordinates)
    correction = (x_soll - coordinates)/splits
    theta_i = theta_0
    for i in range(splits):
        coordinates = coordinates + correction
        positions.append(coordinates)
        print(coordinates)
        theta_i = backward_kinematic_braccio(coordinates, theta_i)
        joints.append(theta_i)
    return joints, positions

In [10]:
x_goal = np.array([0, 100, 100])
theta_0 = np.array([60, 160, 160])
joints, positions = linear_backward(x_goal, theta_0)
for joint in joints:
    print(joint)

[  78.82944746 -116.72322293   10.10090179]
[ 70.94650271 -95.05090064  19.09081161]
[ 63.06355797 -73.37857835  28.08072143]
[ 55.18061322 -51.70625605  37.07063125]
[ 47.29766848 -30.03393376  46.06054107]
[39.41472373 -8.36161147 55.05045089]
[31.53177898 13.31071083 64.04036071]
[23.64883424 34.98303312 73.03027054]
[15.76588949 56.65535541 82.02018036]
[ 7.88294475 78.32767771 91.01009018]
[1.24344979e-14 1.00000000e+02 1.00000000e+02]
[ 65.65340875 155.42730881 165.35557343]
[ 71.39192827 151.31457419 169.33443724]
[ 77.10299823 147.6260059  172.02950135]
[ 82.67635132 144.39137113 173.47285281]
[ 88.01701606 141.68208119 173.66694976]
[ 93.05343302 139.59389314 172.5970924 ]
[ 97.7402928  138.23282696 170.23419655]
[102.05674721 137.70845568 166.52704118]
[106.00177418 138.13971091 161.38013098]
[109.58856932 139.68112081 154.60560638]


In [49]:
forward_kinematic_braccio_fixed([60, 160, 160])[0:3,3]

array([  78.82944746, -116.72322293,   10.10090179])

### Input to Braccio

In [55]:
s = ser.Serial ('COM9', baudrate = 9600, parity = 'N', bytesize = 8, stopbits = 1, timeout = None)

In [44]:
def reset():
    s.write (b'90 b 90 s 90 e 90 v 90 w 60 g\r\n')

In [45]:
def go_to_joint(joint):
    value = f'{int(joint[0])} b 110 s {int(joint[1])} e {int(joint[2])} v 90 w 73 g\r\n'.encode('utf-8')
    s.write (value)
    print(s.read())

In [60]:
reset()

In [57]:
go_to_joint(joints[0])

b'i'


In [58]:
for i, joint in enumerate(joints):
    time.sleep(1)
    go_to_joint(joint)

b'n'
b'i'
b't'
b'i'
b'a'
b'l'
b'i'
b'z'
b'i'
b'n'


In [41]:
print(len(joints))

10


In [14]:
s.write (b'80 b\r\n')

6

In [61]:
s.read(size=64)

b'g, please wait ...\r\ndone\r\n\r\nb: base           90\r\ns: shoulder   '

## Port schliessen

In [63]:
s.close()

### Plot linear movement

In [2]:
import plotly.graph_objects as go
import pandas as pd

In [11]:
data = pd.DataFrame(positions, columns = ['x','y','z'])

In [12]:
positions_braccio = np.array([[281, 0, 0], [281, 0, 37], [281, 0, 100], [281, 0, 162], [281, 0, 260]])
data_braccio = pd.DataFrame(positions_braccio, columns = ['x','y','z'])
data_braccio.head()

,x,y,z
0,281,0,0
1,281,0,37
2,281,0,100
3,281,0,162
4,281,0,260


In [45]:
splot = go.Scatter3d(x=data['x'], y=data['y'], z=data['z'], name='Calculated points for a linear movement')
braccio_plot = go.Scatter3d(x=data_braccio['x'], y=data_braccio['y'], z=data_braccio['z'], name='Braccio')

pdata = [splot, braccio_plot]

layout = go.Layout(
    scene = dict(
        camera=dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=-1.3, y=-1.7, z=0.06)
        ),
        xaxis = dict(range = [0,400], backgroundcolor="rgba(0,0,0,0)"),
        yaxis = dict(range = [-150,150], backgroundcolor="rgba(0,0,0,0)"),
        zaxis = dict(range = [0,265], backgroundcolor="rgba(205,133,63,1)", showticklabels=False),
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

fig = go.Figure(data=pdata, layout=layout)

fig.write_image('image.svg')
fig.show()

In [181]:
!pip install kaleido